In [1]:
from scipy import stats
import numpy as np
import pandas as pd
import pandas_ta as ta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

In [2]:
 # DATASET ##############################################################################
base = pd.read_csv('WDO$_M5_202103101400_202410181825.csv', sep='\t')
base.drop(columns=['<SPREAD>'], inplace=True)
base.rename(columns={
            '<DATE>': 'Date', 
            '<TIME>': 'Time', 
            '<OPEN>': 'Open', 
            '<HIGH>': 'High', 
            '<LOW>' : 'Low', 
            '<CLOSE>': 'Close', 
            '<TICKVOL>':'Tick', 
            '<VOL>': 'Volume'}, inplace = True)
#base['Datetime'] = pd.to_datetime(base['Date']+' '+base['Time'])
base['Datetime'] = base['Date']+' '+base['Time']
base.set_index(base['Date']+' '+base['Time'], inplace=True)
#base=base.tail(228)

In [3]:
def highlight_selection(df, date_column, suggested_date=None):
    #converte a coluna de datas para datetime, se necessário
    df[date_column] = pd.to_datetime(df[date_column])
    
    #Verificar se a data sugerida foi fornecida
    if suggested_date:
        try:
            selected_date = pd.to_datetime(suggested_date, format='%Y.%m.%d')
        except:
            print("Formato de data invalido. Use o Formato 'YYYY.MM.DD'.")
            return None
    else:
        #identificar a ultima data (data_max) se a data sugerida não for fornecida
        selected_date = df[date_column].max()
         
    data_min = df[df[date_column] < selected_date][date_column].max()
    
    #filtar o DataFrame para incluir apenas as linhas entre data_min e data_max
    df_filtered = df[(df[date_column] >= data_min) & (df[date_column] <= selected_date)]
    df_filtered2 = df[df[date_column] == data_min]
    
    return df_filtered, df_filtered2.High.max(), df_filtered2.Low.min()
    

In [4]:
df_base, data_min_high, data_min_low = highlight_selection(base, 'Date') #, '2024.10.15' # insira a data sugerida para avaliação ou deixe em branco para capturar o max

In [5]:
# SUPORTE E RESISTÊNCIA ###########################################################################################
def is_resistance(df,i):
    resistence = (df['High'][i] > df['High'][i-1]
         and df['High'][i] > df['High'][i+1]
         and df['High'][i+1] > df['High'][i+2]
         and df['High'][i-1] > df['High'][i-2])
    return resistence

def is_support(df,i):
    support = (df['Low'][i] < df['Low'][i-1]
         and df['Low'][i] < df['Low'][i+1]
         and df['Low'][i+1] < df['Low'][i+2]
         and df['Low'][i-1] < df['Low'][i-2])
    return support


coordinates = []
for i in range(2, df_base.shape[0] - 2):
    if is_support(df_base, i):
        coordinates.append([
          (df_base.index[i], df_base['Low'][i]), # If support, plot a horizontal line from the low
          (max(df_base.index),df_base['Low'][i])
        ])
    elif is_resistance(df_base, i):
        coordinates.append([
          (df_base.index[i],df_base['High'][i]), # If resistance, plot a horizontal line from the high
          (max(df_base.index),df_base['High'][i])
        ])

#coordinates[:5]
avg_candle_size =  np.mean(df_base['High'] - df_base['Low'])

def is_far_from_level(price, levels, delta):
    is_far = True
    for level in levels:
        if (abs(level - price) < delta):
            is_far = False
            break
    return is_far

no_noise_levels = []
coordinates = []

# We skip the first and the last 2 data points once we need at least 5 candles
# to form the patter
for i in range(2, df_base.shape[0] - 2):

    if is_support(df_base, i):
        price = df_base['Low'][i]
        if is_far_from_level(price, no_noise_levels, avg_candle_size):
            no_noise_levels.append(price)
            coordinates.append([
                (df_base.index[i], price), (max(df_base.index), price)
            ])
            
    elif is_resistance(df_base,i):
        price = df_base['High'][i]
        if is_far_from_level(price, no_noise_levels, avg_candle_size):
            no_noise_levels.append(price)
            coordinates.append([
                (df_base.index[i], price), (max(df_base.index), price)
            ])

In [6]:
c_start = []
c_end = []
c_n_price = []
c_n_price_below = []


for i in range(len(coordinates)):
    c_start.append(coordinates[i][0][0])
    c_end.append(coordinates[i][1][0])
    new_price = coordinates[i][0][1]
    c_n_price.append(new_price)
    new_price_below = coordinates[i][1][1]
    c_n_price_below.append(new_price_below)
    
c_data = {'start' : c_start,
        'end' : c_end,
        'n_price': c_n_price,
        'n_price_below' : c_n_price_below}

df_cordinates = pd.DataFrame(c_data)

In [7]:
# CHANNEL FIMATHE #################################################################################################
# Verifique o preço de abertura do primeiro candle e gere um loop para adicionar ou subtrair o VALOR do channel_size
channel_size = 10.0
df_atual = df_base[df_base["Date"]==df_base['Date'].max()]
first_time_price =  df_atual['Datetime'].iloc[0]
last_time_price =  df_atual['Datetime'].max()
first_open_price = df_atual['Open'].iloc[0]
start = []
end = []
n_price = []
n_price_below = []


for i in range(8):
    start.append(first_time_price)
    end.append(last_time_price)
    new_price = first_open_price + (i * channel_size)
    n_price.append(new_price)
    new_price_below = first_open_price - (i * channel_size)
    n_price_below.append(new_price_below)
    
data = {'start' : start,
        'end' : end,
        'n_price': n_price,
        'n_price_below' : n_price_below}
df_channel = pd.DataFrame(data)

In [8]:
df_base2 = df_base.copy()
df_base2.index = pd.to_datetime(df_base2.Datetime)
df_base2["VWAP"]=ta.vwap(df_base2.High, df_base2.Low, df_base2.Close, df_base2.Volume)
VWAPsignal = [0]*len(df_base2)
backcandles = 15

for row in range(backcandles, len(df_base2)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df_base2.Open[i], df_base2.Close[i])>=df_base2.VWAP[i]:
            dnt=0
        if min(df_base2.Open[i], df_base2.Close[i])<=df_base2.VWAP[i]:
            upt=0
    if upt==1 and dnt==1:
        VWAPsignal[row]=3
    elif upt==1:
        VWAPsignal[row]=2
    elif dnt==1:
        VWAPsignal[row]=1

df_base2['VWAPSignal'] = VWAPsignal
df_base['VWAP'] = df_base2['VWAP']

In [9]:
 #GRAPHS 2 ##############################################################################
price_bins = df_base.copy()
price_bins["Close"] = price_bins["Close"].round(4)
price_bins = price_bins.groupby("Close", as_index=False)["Volume"].sum()
price_bins["Percentage"] = price_bins["Volume"]*100/price_bins["Volume"].sum()

#df_base = df_base.tail(114)

# Moving average over a window of 3 and 8
df_base['moving9'] = df_base['Close'].rolling(9).mean().round(2)
df_base['moving21'] = df_base['Close'].rolling(21).mean().round(2)


candles = make_subplots(rows=2,
                        cols=2,
                        shared_xaxes="columns",
                        shared_yaxes="rows",
                        column_width=[0.8, 0.2],
                        row_heights=[0.8, 0.2],
                        horizontal_spacing=0.05,
                        vertical_spacing=0,
                        specs=[[{"secondary_y": True}, {"secondary_y": True}], [{"colspan": 2}, None]])
    
    
candles.add_trace(go.Candlestick(x = df_base['Datetime'],
                                open = df_base['Open'],
                                high = df_base['High'],
                                low = df_base['Low'],
                                close = df_base['Close'],
                                name = 'WDO'),row = 1, col =1)


candles.update_yaxes(
    showticklabels=True,
    showspikes=True,
    showgrid=False,
    col=1,
    row=1)

# add SMA 9
# adiciona as linhas no grafico de candles
candles.add_trace(go.Scatter(x=df_base['Datetime'],
                       y= df_base['moving9'],
                       mode = 'lines',
                       line_color = 'blue',
                       ), row = 1, col =1) 

# add SMA 21
candles.add_trace(go.Scatter(x=df_base['Datetime'],
                       y= df_base['moving21'],
                       mode = 'lines',
                       line_color = 'green',
                       ), row = 1, col =1) 

# add VWAP (15 periodos)
candles.add_trace(go.Scatter(x=df_base['Datetime'],
                       y= df_base['VWAP'],
                       mode = 'lines',
                       line_color = 'yellow',
                       ), row = 1, col =1) 

#adicionando uma linha fixa com a VWAP do tultimo candle do dia Anterior
candles.add_shape(type="line", x0= df_base.iloc[114][8], 
                            y0 = df_base.iloc[110][9],
                            x1 = df_base.iloc[-1][8],
                            y1 = df_base.iloc[110][9], 
                            fillcolor = 'red',
                            opacity=0.7)


#adicionando uma linha com a maxima e minima do dia Anterior
candles.add_shape(type="line", x0= df_base.iloc[110][8], 
                            y0 = data_min_high,
                            x1 = df_base.iloc[-1][8],
                            y1 = data_min_high, 
                            fillcolor = 'red',
                            opacity=0.7)
candles.add_shape(type="line", x0= df_base.iloc[110][8], 
                            y0 = data_min_low,
                            x1 = df_base.iloc[-1][8],
                            y1 = data_min_low, 
                            fillcolor = 'red',
                            opacity=0.7)




#################################
# Adicionar uma linha em um ponto específico do gráfico
#candles.add_shape(type="line", 
##                  x0='16:30:00', y0=5700,
#                  x1='18:25:00', y1=5700, 
#                  fillcolor='#FF4B4B',
#                  opacity=1)

#for row in df1.iterrows():
#    fig.add_shape(type="line", x0= row[1]['dates'], 
#                            y0 = row[1]['levels'],
#                            x1 = df.date.max(),
#                            y1 = row[1]['levels'], fillcolor = 'yellow')

for row in df_cordinates.iterrows():
    candles.add_shape(type="line", x0= row[1]['start'], 
                            y0 = row[1]['n_price'],
                            x1 = row[1]['end'],
                            y1 = row[1]['n_price'], 
                            fillcolor = 'red',
                            opacity=0.1)


for row in df_channel.iterrows():
    candles.add_shape(type="line", x0= row[1]['start'], 
                            y0 = row[1]['n_price'],
                            x1 = row[1]['end'],
                            y1 = row[1]['n_price'], 
                            fillcolor = 'yellow',
                            opacity=0.5)

for row in df_channel.iterrows():
    candles.add_shape(type="line", x0= row[1]['start'], 
                            y0 = row[1]['n_price_below'],
                            x1 = row[1]['end'],
                            y1 = row[1]['n_price_below'], 
                            fillcolor = 'yellow',
                            opacity=0.5)
#################################


# grafico de volume
candles.add_trace(go.Bar(
    x = df_base['Datetime'],
    y = df_base['Volume'],
    name = 'Volume',
    marker_color = 'white', opacity = 0.13), row =1, col = 1, secondary_y = True)

# grafico de barras lateral com as zonas com concentração de volume
candles.add_trace(
    go.Bar(
        y=price_bins["Close"],
        x=price_bins["Volume"],
        text=price_bins["Percentage"],
        name="Price Bins",
        orientation="h",
        marker_color="purple",
        texttemplate="%{text:.2f}% @ %{y}",
        hoverinfo="x+y"
    ),
    col=2,
    row=1,
)



candles.update_xaxes(
    showticklabels=True,
    showspikes=True,
    showgrid=True,
    col=2,
    row=1)

    
candles.update_layout(xaxis_rangeslider_visible = False, annotations=[dict(x='16:30:00', y=5680.0, xref='x', yref='y',
                        showarrow=False, xanchor='left', text='Canal de Referência')]) #

candles.update_layout(margin=dict(l=5, r=5, t=5, b=5), height=600, showlegend=False, template='plotly_dark') #,title="Meu Dash" 

In [10]:
first_open_price

5644.5

In [18]:
df_channel.shape

(8, 4)

In [30]:
1406+512

1918

In [31]:
1918/22

87.18181818181819

In [11]:
df_channel['n_price']

start                  end  n_price  n_price_below
0  2024.10.18 09:00:00  2024.10.18 18:25:00   5644.5         5644.5
1  2024.10.18 09:00:00  2024.10.18 18:25:00   5654.5         5634.5
2  2024.10.18 09:00:00  2024.10.18 18:25:00   5664.5         5624.5
3  2024.10.18 09:00:00  2024.10.18 18:25:00   5674.5         5614.5
4  2024.10.18 09:00:00  2024.10.18 18:25:00   5684.5         5604.5
5  2024.10.18 09:00:00  2024.10.18 18:25:00   5694.5         5594.5
6  2024.10.18 09:00:00  2024.10.18 18:25:00   5704.5         5584.5
7  2024.10.18 09:00:00  2024.10.18 18:25:00   5714.5         5574.5

In [20]:
precos = list(df_channel['n_price'])

In [21]:
precos

[5644.5, 5654.5, 5664.5, 5674.5, 5684.5, 5694.5, 5704.5, 5714.5]

In [39]:
df_base = df_base.tail(114)

In [40]:
price_channels =  list(df_channel['n_price'])

# Função para rotular os canais
def label_channel(row, channels):
    label = ""
    for i, channel in enumerate(channels):
        if i < len(channels) - 1:
            if row['Close'] > channel and row['Close'] <= channels[i + 1]:
                label = "Canal de Referência"
                break
            elif row['Close'] <= channel:
                label = "Canal de Zona Neutra"
                break
    if label == "":
        label = "Em branco"
    return label

# Aplicar a função para rotular os canais no dataframe
df_base['channel_label'] = df_base.apply(label_channel, axis=1, channels=price_channels)



In [41]:
df_base.channel_label.value_counts()

channel_label
Canal de Referência    114
Name: count, dtype: int64

In [38]:
import pandas as pd

# Criação do dataframe exemplo
data = {
    'close': [5640.0, 5650.0, 5660.0, 5670.0, 5680.0, 5690.0, 5700.0, 5710.0]
}
df = pd.DataFrame(data)

# Lista de preços para os canais
price_channels = [5644.5, 5654.5, 5664.5, 5674.5, 5684.5, 5694.5, 5704.5, 5714.5]

# Função para rotular os canais
def label_channel(row, channels):
    label = ""
    for i, channel in enumerate(channels):
        if i < len(channels) - 1:
            if row['close'] > channel and row['close'] <= channels[i + 1]:
                label = "Canal de Referência"
                break
            elif row['close'] <= channel:
                label = "Canal de Zona Neutra"
                break
    if label == "":
        label = "Em branco"
    return label

# Aplicar a função para rotular os canais no dataframe
df['channel_label'] = df.apply(label_channel, axis=1, channels=price_channels)

print(df)


    close         channel_label
0  5640.0  Canal de Zona Neutra
1  5650.0   Canal de Referência
2  5660.0   Canal de Referência
3  5670.0   Canal de Referência
4  5680.0   Canal de Referência
5  5690.0   Canal de Referência
6  5700.0   Canal de Referência
7  5710.0   Canal de Referência


In [12]:
df_base['Date'].max(), df_base['Date'].min()

(Timestamp('2024-10-18 00:00:00'), Timestamp('2024-10-17 00:00:00'))

In [13]:
df_anterior = df_base[df_base['Date']<df_base['Date'].max()]

In [14]:
def verificar_tamanho_mediano_candles(df):
    df['Tamanho'] = df['High']  - df['Low']
    tamanho_medio = df['Tamanho'].mean()
    return round(tamanho_medio, 2)

In [15]:
print(verificar_tamanho_mediano_candles(df_anterior));

5.12


In [16]:
data_atual = df_base['Date'].max()
data_anterior = df_base['Date'].min()

In [ ]:
cnal = []

In [17]:
#Vou criar um exemplo de código em Python utilizando a biblioteca pandas para manipulação de dados e matplotlib para plotagem dos gráficos.
#
#```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Crie um DataFrame com informações financeiras no padrão OHLC
data = {
    'Open': [5700, 5725, 5750, 5780, 5810],
    'High': [5725, 5750, 5780, 5810, 5835],
    'Low': [5675, 5700, 5725, 5750, 5775],
    'Close': [5730, 5760, 5790, 5820, 5845]
}

df = pd.DataFrame(data)

# Verifique o preço de abertura do primeiro candle e gere um loop para adicionar ou subtrair 5.00 ao valor
first_open_price = df['Open'].iloc[0]

for i in range(20):
    new_price = first_open_price + (i * 5)
    print(f"Preço após {i} ciclos: {new_price}")

# Repita o processo para adicionar 20 linhas para baixo
df_below = df.copy()
for i in range(20):
    new_price_below = first_open_price - (i * 5)
    df_below.loc[i] = [new_price_below, np.nan, np.nan, np.nan]


Preço após 0 ciclos: 5700
Preço após 1 ciclos: 5705
Preço após 2 ciclos: 5710
Preço após 3 ciclos: 5715
Preço após 4 ciclos: 5720
Preço após 5 ciclos: 5725
Preço após 6 ciclos: 5730
Preço após 7 ciclos: 5735
Preço após 8 ciclos: 5740
Preço após 9 ciclos: 5745
Preço após 10 ciclos: 5750
Preço após 11 ciclos: 5755
Preço após 12 ciclos: 5760
Preço após 13 ciclos: 5765
Preço após 14 ciclos: 5770
Preço após 15 ciclos: 5775
Preço após 16 ciclos: 5780
Preço após 17 ciclos: 5785
Preço após 18 ciclos: 5790
Preço após 19 ciclos: 5795
